In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import yaml
import tqdm
import json
import torch
import hydra
import pprint
import logging
import colorlog
import src.prepare  # noqa
import logging.config

import numpy as np
import pytorch_lightning as pl

from src.config import read_config
from hydra.utils import instantiate
from src.load import load_model_from_cfg
from omegaconf import OmegaConf, DictConfig
from hydra import initialize, initialize_config_module, initialize_config_dir, compose

/home/nadir/disk/codes/tmr-code/.tmr.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
logger = logging.getLogger(__name__)

In [6]:
with initialize(
    version_base=None,
    config_path="configs",
    # 
):
    # config = compose(overrides=["+db=mysql"])
    config = compose(
        # config_name="babel-classifier",
        return_hydra_config=True,
        overrides=["+data=babel-classifier"]
    )
    
    from hydra.core.hydra_config import HydraConfig
    
    HydraConfig.instance().set_config(config)
    
print(OmegaConf.to_yaml(config, sort_keys=False))

hydra:
  run:
    dir: outputs/${now:%Y-%m-%d}/${now:%H-%M-%S}
  sweep:
    dir: multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}
    subdir: ${hydra.job.num}
  launcher:
    _target_: hydra._internal.core_plugins.basic_launcher.BasicLauncher
  sweeper:
    _target_: hydra._internal.core_plugins.basic_sweeper.BasicSweeper
    max_batch_size: null
    params: null
  help:
    app_name: ${hydra.job.name}
    header: '${hydra.help.app_name} is powered by Hydra.

      '
    footer: 'Powered by Hydra (https://hydra.cc)

      Use --hydra-help to view Hydra specific help

      '
    template: '${hydra.help.header}

      == Configuration groups ==

      Compose your configuration from those groups (group=option)


      $APP_CONFIG_GROUPS


      == Config ==

      Override anything in the config (foo.bar=value)


      $CONFIG


      ${hydra.help.footer}

      '
  hydra_help:
    template: 'Hydra (${hydra.runtime.version})

      See https://hydra.cc for more info.


      == Flags ==

      

In [7]:
print(f"[config.data.mode]: {config.data.mode}")

dataset = instantiate(config.data, split="all")

print(f"[#dataset]: {len(dataset)}")

[config.data.mode]: classifier
[#dataset]: 52308


In [2]:
for index in dataset:
    pass

SyntaxError: incomplete input (508615472.py, line 1)

In [ ]:
# TODO: make the stats below as plot

In [12]:
# NOTE: moved up here in order to use the segmentation config for the dataset
# defined in the config file rather than the config used to train the model

transitions_lengths = []
actions_lengths = []

for sample in dataset:
    if "transition" in sample["text"]:
        transitions_lengths.append(sample["length"])
    else:
        actions_lengths.append(sample["length"])
        
print(f"[transitions]: {len(transitions_lengths)} ({(len(transitions_lengths) / len(dataset) * 100):.2f}%)")
print(f"[actions]: {len(actions_lengths)} ({(len(actions_lengths) / len(dataset) * 100):.2f}%)")

print("--- --- --- --- --- --- --- --- --- --- --- ---")

print(f"[transitions_durations](s): {sum(transitions_lengths):.2f} ({(sum(transitions_lengths) / (sum(transitions_lengths) + sum(actions_lengths)) * 100):.2f}%)")
print(f"[actions_durations](s): {sum(actions_lengths):.2f} ({(sum(actions_lengths) / (sum(transitions_lengths) + sum(actions_lengths)) * 100):.2f}%)")

print("--- --- --- --- --- --- --- --- --- --- --- ---")

print(f"[transitions_lengths]: {np.min(transitions_lengths)}, {np.average(transitions_lengths)}, {np.max(transitions_lengths)}")

# TODO: checkout what is up with the long transitions
sorted(transitions_lengths)[::]

[transitions]: 15017 (28.71%)
[actions]: 37291 (71.29%)
--- --- --- --- --- --- --- --- --- --- --- ---
[transitions_durations](s): 14787.32 (12.46%)
[actions_durations](s): 103912.87 (87.54%)
--- --- --- --- --- --- --- --- --- --- --- ---
[transitions_lengths]: 0.10000000000000009, 0.9847053339548512, 97.043


[0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000009,
 0.10000000000000053,
 0.10000000000000053,
 0.10000000000000053,
 0.10000000000000053,
 0.10000000000000053,
 0.10000000000000053,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.10000000000000142,
 0.101,
 0.10199999999999987,
 0.10199999999999987,
 0.10200000000000009,
 0.10200000000000031,
 0.10299999999999976,
 0.10300000000000004,
 0.1039999999999992,
 0.10399999999999965,
 0.10399999999999987,
 0.10399999999999998,
 0.10399999999999998,
 0.10400000000000009,
 0.10400000000000009,
 0.10400000000000009,
 0.